# YouTube Trending Videos Analysis & Recommendation System

This notebook provides a comprehensive analysis of YouTube trending videos data from multiple countries and builds an intelligent video recommendation system.

## Contents
1. Data Loading & Merging
2. Data Cleaning & Preprocessing
3. Exploratory Data Analysis
4. Advanced Visualizations
5. Recommendation System
6. Interactive Frontend

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Machine Learning and NLP
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Interactive widgets
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✓ All libraries imported successfully!")

## 2. Load and Merge CSV Files

In [ ]:
# Load all CSV files
print("Loading YouTube data from multiple countries...\n")

ca_data = pd.read_csv('data/CAvideos.csv')
gb_data = pd.read_csv('data/GBvideos.csv')
in_data = pd.read_csv('data/INvideos.csv')
us_data = pd.read_csv('data/USvideos.csv')

# Add country identifier
ca_data['country'] = 'CA'
gb_data['country'] = 'GB'
in_data['country'] = 'IN'
us_data['country'] = 'US'

print(f"CA Videos: {len(ca_data):,} rows")
print(f"GB Videos: {len(gb_data):,} rows")
print(f"IN Videos: {len(in_data):,} rows")
print(f"US Videos: {len(us_data):,} rows")

# Merge all datasets
df = pd.concat([ca_data, gb_data, in_data, us_data], ignore_index=True)

print(f"\n✓ Total merged dataset: {len(df):,} rows")
print(f"✓ Columns: {len(df.columns)}")

# Save merged data
df.to_csv('data/merged_youtube_data.csv', index=False)
print("\n✓ Merged data saved to 'data/merged_youtube_data.csv'")

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Dataset info
print("Dataset Information:")
print("="*50)
df.info()

## 3. Data Cleaning and Preprocessing

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("="*50)
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Handle missing values
df['tags'] = df['tags'].fillna('')
df['description'] = df['description'].fillna('')

# Remove duplicates based on video_id
original_len = len(df)
df = df.drop_duplicates(subset=['video_id'], keep='first')
print(f"Removed {original_len - len(df):,} duplicate videos")

# Clean tags (remove pipe separators)
df['tags'] = df['tags'].apply(lambda x: x.replace('|', ' ') if isinstance(x, str) else '')

# Convert publish_time to datetime
df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')
df['publish_date'] = df['publish_time'].dt.date
df['publish_hour'] = df['publish_time'].dt.hour

# Create engagement metrics
df['engagement_score'] = (df['likes'] + df['comment_count'] * 2 - df['dislikes']) / (df['views'] + 1)
df['like_ratio'] = df['likes'] / (df['likes'] + df['dislikes'] + 1)
df['comment_ratio'] = df['comment_count'] / (df['views'] + 1)

print(f"\n✓ Data cleaned! Final dataset: {len(df):,} unique videos")

## 4. Exploratory Data Analysis

In [ ]:
# Statistical Summary
print("=" * 80)
print("YOUTUBE TRENDING VIDEOS - STATISTICAL SUMMARY")
print("=" * 80)
print(f"\n📊 Total Videos: {len(df):,}")
print(f"🎬 Unique Channels: {df['channel_title'].nunique():,}")
print(f"🌍 Countries: {df['country'].nunique()}")
print(f"📂 Categories: {df['category_id'].nunique()}")
print(f"\n📈 Total Views: {df['views'].sum():,.0f}")
print(f"👍 Total Likes: {df['likes'].sum():,.0f}")
print(f"💬 Total Comments: {df['comment_count'].sum():,.0f}")

print(f"\n📊 Average Statistics per Video:")
print(f"  Views: {df['views'].mean():,.0f}")
print(f"  Likes: {df['likes'].mean():,.0f}")
print(f"  Dislikes: {df['dislikes'].mean():,.0f}")
print(f"  Comments: {df['comment_count'].mean():,.0f}")
print(f"  Engagement Score: {df['engagement_score'].mean():.4f}")

print("\n" + "=" * 80)

In [ ]:
# Top 10 most viewed videos
print("\n🔥 TOP 10 MOST VIEWED VIDEOS")
print("=" * 80)
top_videos = df.nlargest(10, 'views')[['title', 'channel_title', 'views', 'likes', 'country']]
for idx, row in top_videos.iterrows():
    print(f"\n{row['title'][:70]}...")
    print(f"  Channel: {row['channel_title']} | Country: {row['country']}")
    print(f"  Views: {row['views']:,} | Likes: {row['likes']:,}")

In [ ]:
# Country-wise distribution
country_stats = df.groupby('country').agg({
    'video_id': 'count',
    'views': 'sum',
    'likes': 'sum',
    'comment_count': 'sum'
}).round(0)

country_stats.columns = ['Videos', 'Total Views', 'Total Likes', 'Total Comments']
print("\n🌍 COUNTRY-WISE STATISTICS")
print("=" * 80)
print(country_stats)

## 5. Data Visualizations

Let's create comprehensive and informative visualizations to understand the data better.

In [ ]:
# 1. Views Distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogram
axes[0].hist(np.log10(df['views'] + 1), bins=50, color='#FF6B6B', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Log10(Views)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('Distribution of Video Views (Log Scale)', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Box plot by country
df.boxplot(column='views', by='country', ax=axes[1], patch_artist=True)
axes[1].set_xlabel('Country', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Views', fontsize=12, fontweight='bold')
axes[1].set_title('Views Distribution by Country', fontsize=14, fontweight='bold')
axes[1].set_yscale('log')
plt.suptitle('')

plt.tight_layout()
plt.show()

In [ ]:
# 2. Engagement Metrics Comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Likes vs Views
axes[0, 0].scatter(np.log10(df['views'] + 1), np.log10(df['likes'] + 1), 
                   alpha=0.3, c='#4ECDC4', s=10)
axes[0, 0].set_xlabel('Log10(Views)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Log10(Likes)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Likes vs Views Relationship', fontsize=13, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Comments vs Views
axes[0, 1].scatter(np.log10(df['views'] + 1), np.log10(df['comment_count'] + 1), 
                   alpha=0.3, c='#FF6B9D', s=10)
axes[0, 1].set_xlabel('Log10(Views)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Log10(Comments)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Comments vs Views Relationship', fontsize=13, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Engagement Score Distribution
axes[1, 0].hist(df['engagement_score'], bins=50, color='#95E1D3', edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Engagement Score', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Distribution of Engagement Scores', fontsize=13, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Like Ratio Distribution
axes[1, 1].hist(df['like_ratio'], bins=50, color='#FFE66D', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Like Ratio', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Distribution of Like Ratios', fontsize=13, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 3. Country-wise Performance Analysis
country_data = df.groupby('country').agg({
    'views': 'sum',
    'likes': 'sum',
    'comment_count': 'sum',
    'video_id': 'count'
}).reset_index()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Total Views by Country
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
axes[0, 0].bar(country_data['country'], country_data['views'], color=colors, edgecolor='black')
axes[0, 0].set_xlabel('Country', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Total Views', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Total Views by Country', fontsize=14, fontweight='bold')
axes[0, 0].ticklabel_format(style='plain', axis='y')
for i, v in enumerate(country_data['views']):
    axes[0, 0].text(i, v, f'{v/1e9:.2f}B', ha='center', va='bottom', fontweight='bold')

# Total Likes by Country
axes[0, 1].bar(country_data['country'], country_data['likes'], color=colors, edgecolor='black')
axes[0, 1].set_xlabel('Country', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Total Likes', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Total Likes by Country', fontsize=14, fontweight='bold')
for i, v in enumerate(country_data['likes']):
    axes[0, 1].text(i, v, f'{v/1e6:.1f}M', ha='center', va='bottom', fontweight='bold')

# Video Count by Country
axes[1, 0].bar(country_data['country'], country_data['video_id'], color=colors, edgecolor='black')
axes[1, 0].set_xlabel('Country', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Number of Videos', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Video Count by Country', fontsize=14, fontweight='bold')
for i, v in enumerate(country_data['video_id']):
    axes[1, 0].text(i, v, f'{v:,}', ha='center', va='bottom', fontweight='bold')

# Pie chart - Country Distribution
axes[1, 1].pie(country_data['video_id'], labels=country_data['country'], autopct='%1.1f%%',
               colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1, 1].set_title('Video Distribution by Country', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# 4. Top Categories Analysis
category_stats = df.groupby('category_id').agg({
    'video_id': 'count',
    'views': 'mean',
    'likes': 'mean'
}).sort_values('video_id', ascending=False).head(10)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top categories by video count
axes[0].barh(category_stats.index.astype(str), category_stats['video_id'], 
             color='#FF6B6B', edgecolor='black', alpha=0.8)
axes[0].set_xlabel('Number of Videos', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Category ID', fontsize=12, fontweight='bold')
axes[0].set_title('Top 10 Categories by Video Count', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Average views by category
top_view_cats = df.groupby('category_id')['views'].mean().sort_values(ascending=False).head(10)
axes[1].barh(top_view_cats.index.astype(str), top_view_cats.values, 
             color='#4ECDC4', edgecolor='black', alpha=0.8)
axes[1].set_xlabel('Average Views', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Category ID', fontsize=12, fontweight='bold')
axes[1].set_title('Top 10 Categories by Average Views', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# 5. Publishing Time Analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Videos by Hour of Day
hour_counts = df['publish_hour'].value_counts().sort_index()
axes[0].plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, 
             markersize=8, color='#FF6B6B')
axes[0].fill_between(hour_counts.index, hour_counts.values, alpha=0.3, color='#FF6B6B')
axes[0].set_xlabel('Hour of Day', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Videos', fontsize=12, fontweight='bold')
axes[0].set_title('Video Publishing Pattern by Hour', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].set_xticks(range(0, 24, 2))

# Average views by hour
hour_views = df.groupby('publish_hour')['views'].mean()
axes[1].bar(hour_views.index, hour_views.values, color='#4ECDC4', edgecolor='black', alpha=0.8)
axes[1].set_xlabel('Hour of Day', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Views', fontsize=12, fontweight='bold')
axes[1].set_title('Average Views by Publishing Hour', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(0, 24, 2))
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 6. Correlation Heatmap
fig, ax = plt.subplots(figsize=(12, 8))

# Select numeric columns for correlation
numeric_cols = ['views', 'likes', 'dislikes', 'comment_count', 'engagement_score', 'like_ratio']
correlation_matrix = df[numeric_cols].corr()

sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=2, cbar_kws={"shrink": 0.8},
            annot_kws={'fontsize': 11, 'fontweight': 'bold'})

plt.title('Correlation Matrix of Video Metrics', fontsize=16, fontweight='bold', pad=20)
plt.xticks(fontsize=11, fontweight='bold')
plt.yticks(fontsize=11, fontweight='bold', rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# 7. Word Cloud from Video Titles
print("Generating Word Cloud from Video Titles...")

# Combine all titles
all_titles = ' '.join(df['title'].astype(str).values)

# Create word cloud
wordcloud = WordCloud(width=1600, height=800, background_color='white', 
                      colormap='viridis', max_words=100, 
                      relative_scaling=0.5, min_font_size=10).generate(all_titles)

plt.figure(figsize=(16, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Common Words in Video Titles', fontsize=18, fontweight='bold', pad=20)
plt.tight_layout(pad=0)
plt.show()

print("✓ Word cloud generated!")

## 6. Build Video Recommendation System

Now let's build a content-based recommendation system using TF-IDF and cosine similarity.

In [ ]:
# Prepare features for recommendation
print("Building recommendation system...")

# Combine text features
df['combined_features'] = (
    df['title'].fillna('') + ' ' + 
    df['tags'] + ' ' + 
    df['channel_title'].fillna('') + ' ' +
    df['description'].apply(lambda x: str(x)[:200] if pd.notna(x) else '')
)

# Calculate popularity score
df['popularity_score'] = (
    np.log1p(df['views']) * 0.4 +
    np.log1p(df['likes']) * 0.3 +
    np.log1p(df['comment_count']) * 0.3
)

print("✓ Features prepared!")
print(f"✓ Dataset size: {len(df):,} videos")

In [ ]:
# Create TF-IDF Matrix
print("Creating TF-IDF matrix...")

tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2
)

tfidf_matrix = tfidf.fit_transform(df['combined_features'])

print(f"✓ TF-IDF matrix created!")
print(f"  Shape: {tfidf_matrix.shape}")
print(f"  Features: {len(tfidf.get_feature_names_out())}")

In [ ]:
# Recommendation Function
def get_video_recommendations(video_id=None, title=None, n_recommendations=10):
    """
    Get video recommendations based on video_id or title
    """
    # Find video index
    if video_id:
        idx = df[df['video_id'] == video_id].index
    elif title:
        idx = df[df['title'].str.contains(title, case=False, na=False)].index
    else:
        print("❌ Please provide either video_id or title")
        return None
    
    if len(idx) == 0:
        print("❌ Video not found!")
        return None
    
    idx = idx[0]
    
    # Get the video details
    video = df.iloc[idx]
    print("="*80)
    print(f"📹 SELECTED VIDEO")
    print("="*80)
    print(f"Title: {video['title']}")
    print(f"Channel: {video['channel_title']}")
    print(f"Views: {video['views']:,} | Likes: {video['likes']:,} | Comments: {video['comment_count']:,}")
    print(f"Country: {video['country']} | Category: {video['category_id']}")
    
    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[idx:idx+1], tfidf_matrix).flatten()
    
    # Get similar video indices
    similar_indices = cosine_sim.argsort()[::-1][1:n_recommendations+1]
    
    # Create recommendations dataframe
    recommendations = df.iloc[similar_indices][['title', 'channel_title', 'views', 
                                                  'likes', 'comment_count', 'country', 
                                                  'category_id']].copy()
    recommendations['similarity_score'] = cosine_sim[similar_indices]
    
    print(f"\n{'='*80}")
    print(f"🎯 TOP {n_recommendations} RECOMMENDATIONS")
    print("="*80)
    
    for i, (idx, row) in enumerate(recommendations.iterrows(), 1):
        print(f"\n{i}. {row['title'][:70]}...")
        print(f"   Channel: {row['channel_title']}")
        print(f"   Views: {row['views']:,} | Likes: {row['likes']:,} | Country: {row['country']}")
        print(f"   Similarity: {row['similarity_score']:.4f}")
    
    return recommendations

print("✓ Recommendation function ready!")

## 7. Test the Recommendation System

In [ ]:
# Test with a popular video
test_video_id = df.nlargest(1, 'views').iloc[0]['video_id']
recommendations = get_video_recommendations(video_id=test_video_id, n_recommendations=5)

In [ ]:
# Test with search by title
recommendations = get_video_recommendations(title='Music', n_recommendations=5)

## 8. Interactive Frontend with Widgets

Let's create a simple and clean interactive interface for the recommendation system.

In [ ]:
# Create interactive widgets
def display_recommendations(video_selection, num_recommendations):
    """Display recommendations in a clean format"""
    output_area.clear_output(wait=True)
    
    with output_area:
        # Get recommendations
        if video_selection:
            recommendations = get_video_recommendations(
                video_id=video_selection, 
                n_recommendations=num_recommendations
            )

# Prepare video selection options (top 100 popular videos)
popular_videos = df.nlargest(100, 'popularity_score')
video_options = {f"{row['title'][:60]}... ({row['channel_title']})": row['video_id'] 
                 for _, row in popular_videos.iterrows()}

# Create widgets
style = {'description_width': '150px'}

video_dropdown = widgets.Dropdown(
    options=video_options,
    description='Select Video:',
    style=style,
    layout=widgets.Layout(width='800px')
)

num_slider = widgets.IntSlider(
    value=5,
    min=3,
    max=15,
    step=1,
    description='Recommendations:',
    style=style,
    layout=widgets.Layout(width='400px')
)

recommend_button = widgets.Button(
    description='🎯 Get Recommendations',
    button_style='success',
    layout=widgets.Layout(width='250px', height='40px'),
    style={'font_weight': 'bold'}
)

output_area = widgets.Output()

def on_button_click(b):
    display_recommendations(video_dropdown.value, num_slider.value)

recommend_button.on_click(on_button_click)

# Display interface
print("="*80)
print("🎬 YOUTUBE VIDEO RECOMMENDATION SYSTEM")
print("="*80)
print("\nSelect a video from the dropdown and click the button to get recommendations!")
print("\n")

display(widgets.VBox([
    widgets.HTML('<h3 style="color: #FF0000;">YouTube Video Recommendation System</h3>'),
    widgets.HTML('<p style="color: #555;">Select a video and get personalized recommendations based on content similarity</p>'),
    video_dropdown,
    num_slider,
    recommend_button,
    output_area
], layout=widgets.Layout(padding='20px', border='2px solid #ddd', border_radius='10px')))

## Summary

This notebook has successfully:

1. ✅ **Merged** four YouTube trending video datasets (CA, GB, IN, US) into a single comprehensive dataset
2. ✅ **Cleaned and preprocessed** the data, handling missing values and creating derived metrics
3. ✅ **Analyzed** the data with comprehensive statistical summaries
4. ✅ **Visualized** the data with informative charts covering:
   - View distributions and engagement metrics
   - Country-wise performance comparisons
   - Category analysis
   - Publishing time patterns
   - Correlation matrices
   - Word clouds
5. ✅ **Built** a content-based recommendation system using TF-IDF and cosine similarity
6. ✅ **Created** an interactive frontend with widgets for easy video recommendations

The recommendation system can suggest similar videos based on title, tags, channel, and description content, helping users discover relevant content!